# Введение в MapReduce модель на Python


In [2]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [3]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [4]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [5]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=33, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [6]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [7]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=33, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [8]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [9]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(33, User(id=1, age=33, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [10]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [11]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(33,
  [User(id=1, age=33, social_contacts=240, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')]),
 (25, [User(id=2, age=25, social_contacts=500, gender='female')])]

In [12]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(33, 520.0), (25, 500.0)]

Все действия одним конвейером!

In [13]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(33, 520.0), (25, 500.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [14]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [15]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [16]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j]) # Значение [i, j] элемента * значение vec[j]. Для каждой строки i образуется shape[1] строк подобного формата

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum) #shape[1] строк, полученных с помощью MAP объединяются с помощью суммирования

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j]) #Представление матрицы. Сохранение [i, j] элемента в формате ((i, j), key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.695821756819591),
 (1, 1.695821756819591),
 (2, 1.695821756819591),
 (3, 1.695821756819591),
 (4, 1.695821756819591)]

## Inverted index

In [17]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document) #Индекс строки - строка

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId) # разбивает строки на слова и создаёт комбинацию слово - номер строки

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds)) # объединяет одинаковые слова и выводит в каких строках они появлялись

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [18]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line) #{номер элемента списка}:{Номер строки в элементе списка}, значение строки

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [19]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [20]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT(): #Разделяет RECORDREADER() на maps штук
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=REDUCE)
#Если указать COMBINER=REDUCE, то работа произойдёт быстрее из-за того, что Комбинатор работает над всеми задачами перед распределением на всех производителей
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

16 key-value pairs were sent over a network.


[(0, [('', 6), ('what', 10)]),
 (1, [('a', 2), ('banana', 2), ('is', 18), ('it', 18)])]

## TeraSort

In [21]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.06668739290005621),
   (None, 0.09060657927614968),
   (None, 0.10230431344999413),
   (None, 0.10748040985647511),
   (None, 0.12194508720344632),
   (None, 0.14876385755317345),
   (None, 0.23634823652895898),
   (None, 0.24555012140775467),
   (None, 0.24771060719434412),
   (None, 0.2641873648777022),
   (None, 0.2716724952843237),
   (None, 0.37501657190366056),
   (None, 0.3976341307308945),
   (None, 0.40178806298516934),
   (None, 0.4902665936747498)]),
 (1,
  [(None, 0.5096660622526463),
   (None, 0.5181926312280886),
   (None, 0.5397621150389451),
   (None, 0.587863310897615),
   (None, 0.6133152103451096),
   (None, 0.6413332265001567),
   (None, 0.6695373947883921),
   (None, 0.7515344351064858),
   (None, 0.8305670681715228),
   (None, 0.8524263042228745),
   (None, 0.864336913127988),
   (None, 0.8788305195876194),
   (None, 0.9256961140740451),
   (None, 0.9383159458317832),
   (None, 0.974493582803867)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [22]:
import numpy as np

input_val = np.random.rand(30)

def MAP(_, value):
  yield (None, value)

def REDUCE(_, value):
  yield max(value)

def RECORDREADER():
  for i in range(input_val.shape[0]):
    yield (None, input_val[i])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[0.9903948549357116]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [23]:
input_val = np.random.rand(30)

def MAP(_, value):
  yield (None, value)

def REDUCE(_, value):
  yield sum(value) / len(value)

def RECORDREADER():
  for i in range(input_val.shape[0]):
    yield (None, input_val[i])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[0.4754923839696078]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [24]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
    t[k2] = sorted(t[k2])
  return t.items()

input_list = [(1, 'a'), (2, 'd'), (1, 'c'), (2, 'b'), (3, 'e')]
output = groupbykey(input_list)
print(output)

(list(groupbykey(map_output)))


dict_items([(1, ['a', 'c']), (2, ['b', 'd']), (3, ['e'])])


[(33,
  [User(id=1, age=33, social_contacts=240, gender='female'),
   User(id=3, age=33, social_contacts=800, gender='female')]),
 (25, [User(id=2, age=25, social_contacts=500, gender='female')])]

### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [25]:
input_collection = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=1, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753),
]

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for userindex, user in enumerate(split):
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])


def MAP(_, user: str):
    yield (user.id, user)


def REDUCE(user_id: int, users: Iterator[int]): # (33, [User(id=1, age=33, social_contacts=240, gender='female'), User(id=3, age=33, social_contacts=800, gender='female')])
    yield users[0]                              # Оставляем только нулевой элемент


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

7 key-value pairs were sent over a network.


[(0, [User(id=0, age=25, social_contacts=540, gender='female')]),
 (1,
  [User(id=1, age=16, social_contacts=123, gender='male'),
   User(id=2, age=54, social_contacts=120, gender='male'),
   User(id=3, age=27, social_contacts=742, gender='male')])]

#Операторы реляционной алгебры

### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [26]:
#Select * from users where gender="female" and age > 30;
input_collection = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=1, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753),
]


def MAP(_, row:NamedTuple):
  if (row.gender == 'female' and row.age > 30):
    yield (row, row)

def REDUCE_SELECT(_, rows: NamedTuple):
    yield (rows, rows)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE_SELECT)
output = list(output)
output

[([User(id=2, age=35, social_contacts=297, gender='female')],
  [User(id=2, age=35, social_contacts=297, gender='female')]),
 ([User(id=1, age=32, social_contacts=753, gender='female')],
  [User(id=1, age=32, social_contacts=753, gender='female')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [27]:
#Проекция по столбцу gender
input_collection = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=1, age=16, gender="male", social_contacts=123),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753),
]

def MAP(_, row:NamedTuple):
    yield (row.gender, row.gender)

def REDUCE(row, rows: NamedTuple):
    yield (row, rows[0])

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('male', 'male'), ('female', 'female')]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [28]:
input_collection_a = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

input_collection_b = [
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753)
]

def MAP(_, row:NamedTuple):
    yield (row, row)

def REDUCE(row, rows: NamedTuple):
    yield (row, rows[0])

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=2, age=54, social_contacts=120, gender='male'),
  User(id=2, age=54, social_contacts=120, gender='male')),
 (User(id=0, age=25, social_contacts=540, gender='female'),
  User(id=0, age=25, social_contacts=540, gender='female')),
 (User(id=3, age=27, social_contacts=742, gender='male'),
  User(id=3, age=27, social_contacts=742, gender='male')),
 (User(id=3, age=31, social_contacts=521, gender='male'),
  User(id=3, age=31, social_contacts=521, gender='male')),
 (User(id=2, age=35, social_contacts=297, gender='female'),
  User(id=2, age=35, social_contacts=297, gender='female')),
 (User(id=1, age=16, social_contacts=123, gender='male'),
  User(id=1, age=16, social_contacts=123, gender='male')),
 (User(id=1, age=32, social_contacts=753, gender='female'),
  User(id=1, age=32, social_contacts=753, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [29]:
input_collection_a = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

input_collection_b = [
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753)
]

def MAP(_, row:NamedTuple):
    yield (row, row)

def REDUCE(row, rows: NamedTuple):
  if len(rows) == 2:
    yield (row, rows[0])

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=3, age=31, social_contacts=521, gender='male'),
  User(id=3, age=31, social_contacts=521, gender='male')),
 (User(id=2, age=35, social_contacts=297, gender='female'),
  User(id=2, age=35, social_contacts=297, gender='female'))]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [30]:
input_collection_a = [
    User(id=2, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

input_collection_b = [
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=3, age=31, gender="male", social_contacts=521),
    User(id=1, age=32, gender="female", social_contacts=753)
]
def MAP(_, row:NamedTuple):
  if row in input_collection_b:
    yield (row, 'S')
  if row in input_collection_a:
    yield (row, 'R')

def REDUCE(row, charic):
  if charic[0] == 'R':
    yield (row, row)

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(User(id=2, age=54, social_contacts=120, gender='male'),
  User(id=2, age=54, social_contacts=120, gender='male')),
 (User(id=0, age=25, social_contacts=540, gender='female'),
  User(id=0, age=25, social_contacts=540, gender='female')),
 (User(id=3, age=27, social_contacts=742, gender='male'),
  User(id=3, age=27, social_contacts=742, gender='male')),
 (User(id=3, age=31, social_contacts=521, gender='male'),
  User(id=3, age=31, social_contacts=521, gender='male')),
 (User(id=2, age=35, social_contacts=297, gender='female'),
  User(id=2, age=35, social_contacts=297, gender='female')),
 (User(id=1, age=16, social_contacts=123, gender='male'),
  User(id=1, age=16, social_contacts=123, gender='male'))]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [31]:
input_collection_a = [
    User(id=5, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=4, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

class User_Tel(NamedTuple):
  id: int
  tel: int
  info: str

input_collection_b = [
    User_Tel(id=2, tel=763672, info="None"),
    User_Tel(id=3, tel=899080, info="None"),
    User_Tel(id=1, tel=708090, info="None"),
    User_Tel(id=10, tel=708090, info="None"),
]

def MAP(_, row:NamedTuple):
  if row in input_collection_a and row.id in [r.id for r in input_collection_b]:
    yield (row.id, ('R', (row.age, row.gender, row.social_contacts)))
  elif row in input_collection_b and row.id in [r.id for r in input_collection_a]:
    yield (row.id, ('S', (row.tel, row.info)))


def REDUCE(row, charic):
  for i in range(len(charic)):
    for j in range(len(charic)):
      if charic[i][0] == 'R' and charic[j][0] == 'S':
        yield (None, (charic[i][1], row, charic[j][1]))

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None, ((27, 'male', 742), 3, (899080, 'None'))),
 (None, ((35, 'female', 297), 2, (763672, 'None'))),
 (None, ((16, 'male', 123), 1, (708090, 'None')))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [32]:
input_collection_a = [
    User(id=5, age=54, gender="male", social_contacts=120),
    User(id=0, age=25, gender="female", social_contacts=540),
    User(id=3, age=27, gender="male", social_contacts=742),
    User(id=4, age=31, gender="male", social_contacts=521),
    User(id=2, age=35, gender="female", social_contacts=297),
    User(id=1, age=16, gender="male", social_contacts=123)
]

def MAP(_, row:NamedTuple):
  yield (row.gender, row.social_contacts)

def REDUCE(row, charic): #GROUP BY USER.GENDER HAVING MAX(USER.SOCIAL_CONTACTS)
  yield (row, max(charic))

def RECORDREADER():
  return [(u.id, u) for u in input_collection_a]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('male', 742), ('female', 540)]

# Работа над матрицами

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [33]:
from typing import Iterator
import numpy as np
n = 4
mat = np.ones((5,n))
vec1 = np.random.rand(n // 2) # in-memory vector in all map tasks
vec2 = np.random.rand(n - n // 2) #Подразумевается разбиение ультрабольшого вектора на вектора поменьше, которые уже помещаются в памяти

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  if(j < len(vec1)):
    yield (i, value*vec1[j]) # Значение [i, j] элемента * значение vec[j]. Для каждой строки i образуется shape[1] строк подобного формата
  else:
    yield (i, value*vec2[j-len(vec1)]) # Значение [i, j] элемента * значение vec[j]. Для каждой строки i образуется shape[1] строк подобного формата

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum) #shape[1] строк, полученных с помощью MAP объединяются с помощью суммирования

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j]) #Представление матрицы. Сохранение [i, j] элемента в формате ((i, j), key)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.7554091947556474),
 (1, 2.7554091947556474),
 (2, 2.7554091947556474),
 (3, 2.7554091947556474),
 (4, 2.7554091947556474)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [34]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [35]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  for i in range(small_mat.shape[0]):
    yield ((i, k), small_mat[i][j] * v1)

def REDUCE(key, values):
  yield  (key, sum(values))


Проверьте своё решение

In [36]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [37]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [38]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for i in range (small_mat.shape[0]):
    for j in range(big_mat.shape[0]):
      for k in range(big_mat.shape[1]):
        yield (((i, j), small_mat[i,j]), ((j, k), big_mat[j,k]))


def MAP(mat1, mat2):
  (k1, v1) = mat1
  (k2, v2) = mat2
  (i, _) = k1
  (_, k) = k2
  yield ((i, k), v1 * v2)

def REDUCE(key, values):
  yield (key, sum(values))

# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [40]:
import numpy as np

I = 2
J = 3
K = 4*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
  def RECORDREADER(key, split):
    mat = []
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        mat.append(((key, i, j), split[i, j]))
    return mat

  list1 = RECORDREADER('S', small_mat)
  list2 = RECORDREADER('B', big_mat)
  yield list1
  yield list2


def MAP(k, v):
  (mat, i, j) = k
  w = v
  if mat == 'S':
    yield (j, (mat, i, w))
  else:
    yield (i, (mat, j, w))

def REDUCE(key, values):
  small = [v for v in values if v[0] == 'S']
  big = [v for v in values if v[0] == 'B']
  for s in small:
    for b in big:
      yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


In [41]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [42]:
import numpy as np

I = 2
J = 3
K = 4*10

maps = 2
reducers = 3

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def INPUTFORMAT():
  global maps
  def RECORDREADER(key, split):
    mat = []
    for i in range(split.shape[0]):
      for j in range(split.shape[1]):
        mat.append(((key, i, j), split[i, j]))
    return mat

  small = RECORDREADER('S', small_mat)
  split_size =  int(np.ceil(len(small)/maps))
  for i in range(0, len(small), split_size):
    yield small[i:i+split_size]

  big = RECORDREADER('B', big_mat)
  split_size =  int(np.ceil(len(big)/maps))
  for i in range(0, len(big), split_size):
    yield big[i:i+split_size]

def MAP(k, v):
  (mat, i, j) = k
  w = v
  if mat == 'S':
    yield (j, (mat, i, w))
  else:
    yield (i, (mat, j, w))

def REDUCE(key, values):
  small = [v for v in values if v[0] == 'S']
  big = [v for v in values if v[0] == 'B']
  for s in small:
    for b in big:
      yield ((s[1], b[1]), s[2] * b[2])

def INPUT_MUL():
  for j in joined:
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]

mul_output = MapReduceDistributed(INPUT_MUL, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

126 key-value pairs were sent over a network.
240 key-value pairs were sent over a network.


In [43]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True